In [2]:
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.applications.inception_v3 import InceptionV3 as PretrainedModel, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_path = './data/train'
valid_path = './data/test'
train_image_files = glob(train_path +'/*/*.jpg')
valid_image_files = glob(valid_path +'/*/*.jpg')

In [3]:
folders = glob(train_path + '/*')
print(folders)

['./data/train\\melanoma', './data/train\\no_melanoma']


In [4]:
IMAGE_SIZE = [150, 150]
ptm = PretrainedModel(
    input_shape=IMAGE_SIZE+[3],
    weights='imagenet',
    include_top=False)

#freeze pre-trained model weights
ptm.trainable = False

In [5]:
#map the data into features vectors
K = len(folders) #number of classes
x = Flatten()(ptm.output)
#x = Dense(K,activation='softmax')(x)

model = Model(inputs=ptm.input, outputs=x)

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [7]:
#Generador para el conjunto de entrenamiento
batch_size = 16

#Se crea un Image Generator 
gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = gen.flow_from_directory(
    train_path,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    class_mode = 'binary'
)
# Generador para el conjunto de validación
valid_generator = gen.flow_from_directory(
    valid_path,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    class_mode = 'binary'
)

Found 6916 images belonging to 2 classes.
Found 1729 images belonging to 2 classes.


In [8]:
N_train = len(train_image_files)
N_valid = len(valid_image_files)

#figure output size
feat = model.predict(np.random.random([1]+IMAGE_SIZE+[3]))
D = feat.shape[1]

X_train = np.zeros((N_train,D))
Y_train = np.zeros(N_train)
X_valid = np.zeros((N_valid,D))
Y_valid = np.zeros(N_valid)

In [ ]:
i = 0
for x, y in train_generator:
    D = model.predict(x)
     #get features
    features = model.predict(x)
  
    #size of the batch (not always is the size of the batch)
    sz = len(y) #nuber of samples on this batch
    X_train[i:i+sz]=features
    Y_train[i:i+sz]=y

    i += sz
    if i%1280==0:
        print(i)

    if i>=N_train:
        print('breaking now')
        break

print(i)

In [10]:
np.save('X_train_InceptionV3.npy', X_train)
np.save('Y_train_InceptionV3.npy', Y_train)

In [11]:
i = 0
for x, y in valid_generator:
    D = model.predict(x)
     #get features
    features = model.predict(x)
  
    #size of the batch (not always is the size of the batch)
    sz = len(y) #nuber of samples on this batch
    X_valid[i:i+sz]=features
    Y_valid[i:i+sz]=y

    i += sz
    if i%1280==0:
        print(i)

    if i>=N_valid:
        print('breaking now')
        break

print(i)

1280
breaking now
1729


In [12]:
np.save('X_valid_InceptionV3.npy', X_valid)
np.save('Y_valid_InceptionV3.npy', Y_valid)

In [9]:
X_train = np.load('X_train_InceptionV3.npy')
Y_train = np.load('Y_train_InceptionV3.npy')
X_valid = np.load('X_valid_InceptionV3.npy')
Y_valid = np.load('Y_valid_InceptionV3.npy')

In [10]:
import tensorflow as tf

METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
]

feat = model.predict(np.random.random([1]+IMAGE_SIZE+[3]))
D = feat.shape[1]

#Modelo de red neuronal para clasificación binaria
in3 = Input(shape=(D,))
x = Dense(2048,activation='relu')(in3)
x = Dropout(0.2)(x)
x = Dense(1024,activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(512,activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1,activation='sigmoid')(x)

mela_model = Model(in3,x)

mela_model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=METRICS
)

In [11]:
r3 = mela_model.fit(
    X_train, Y_train,
    batch_size=128,
    epochs=20,
    validation_data=(X_valid,Y_valid)
)

Epoch 1/20
55/55 [==============================] - 7s 91ms/step - loss: 1.6345 - tp: 2258.0000 - fp: 1326.0000 - tn: 1946.0000 - fn: 1386.0000 - accuracy: 0.6079 - precision: 0.6300 - recall: 0.6196 - auc: 0.6259 - val_loss: 0.5467 - val_tp: 769.0000 - val_fp: 346.0000 - val_tn: 472.0000 - val_fn: 142.0000 - val_accuracy: 0.7178 - val_precision: 0.6897 - val_recall: 0.8441 - val_auc: 0.8168
Epoch 2/20
55/55 [==============================] - 4s 66ms/step - loss: 0.4767 - tp: 2865.0000 - fp: 749.0000 - tn: 2523.0000 - fn: 779.0000 - accuracy: 0.7791 - precision: 0.7928 - recall: 0.7862 - auc: 0.8565 - val_loss: 0.3740 - val_tp: 783.0000 - val_fp: 146.0000 - val_tn: 672.0000 - val_fn: 128.0000 - val_accuracy: 0.8415 - val_precision: 0.8428 - val_recall: 0.8595 - val_auc: 0.9163
Epoch 3/20
55/55 [==============================] - 4s 65ms/step - loss: 0.3021 - tp: 3106.0000 - fp: 303.0000 - tn: 2969.0000 - fn: 538.0000 - accuracy: 0.8784 - precision: 0.9111 - recall: 0.8524 - auc: 0.9387 

In [12]:
test = pd.read_csv('sample.csv')
test['ID']=test['ID']+'.jpg'
test.head()

,ID,riesgo
0,c890196d33d6b569680c6f089bc83b95.jpg,0.5
1,93ead62bbb22ed4ef466d308fd32476e.jpg,0.5
2,752ba789481175a51f5f986edae665ac.jpg,0.5
3,d6428bef7cf365baaeab1584f65ac80b.jpg,0.5
4,baf3e9e642184e461cf9f7f5b9c0d06e.jpg,0.5


In [13]:
path = './test/test/'
import cv2
prob = []
for file in list(test['ID']):
    img = cv2.imread(path+file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    res = cv2.resize(img, dsize=(150,150))
    x = image.img_to_array(res)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    feat = model.predict(x)
    mela = mela_model.predict(feat)
    #print(mela)
    prob.append(mela[0][0])

In [14]:
test_2 = pd.read_csv('sample.csv')
test_2['riesgo'] = prob
test_2.tail(5)

,ID,riesgo
9933,a2ad05042cc442972546c81ebf564d17,0.920635
9934,1cfc9c0844fc8c39c6db211fb57a4b00,1.000000
9935,4e4deb3e76c2f32011f0c54a83864571,0.999985
9936,d4d51aaef0b34007705be379219f56f3,1.000000
9937,48ca1384ceab3f5261dedf80196cef9b,0.002461


In [16]:
test_2['riesgo']=1-test_2['riesgo']

In [17]:
test_2.tail(5)
test_2.to_csv('test_inceptionv3.csv', index=False)

In [27]:
mela_model.save('inceptionv3_model_150x150.h5')